In [1]:
import numpy as np
import pandas as pd

In [2]:
data = pd.read_parquet('/kaggle/input/original-data/original.parquet')

In [3]:
data.head()

,session_id,index,elapsed_time,event_name,name,level,page,room_coor_x,room_coor_y,screen_coor_x,screen_coor_y,hover_duration,text,fqid,room_fqid,text_fqid,fullscreen,hq,music,level_group
0,20090312431273200,0,0,cutscene_click,basic,0,NaN,-413.991405,-159.314686,380.0,494.0,NaN,undefined,intro,tunic.historicalsociety.closet,tunic.historicalsociety.closet.intro,0,0,1,0-4
1,20090312431273200,1,1323,person_click,basic,0,NaN,-413.991405,-159.314686,380.0,494.0,NaN,"Whatcha doing over there, Jo?",gramps,tunic.historicalsociety.closet,tunic.historicalsociety.closet.gramps.intro_0_...,0,0,1,0-4
2,20090312431273200,2,831,person_click,basic,0,NaN,-413.991405,-159.314686,380.0,494.0,NaN,Just talking to Teddy.,gramps,tunic.historicalsociety.closet,tunic.historicalsociety.closet.gramps.intro_0_...,0,0,1,0-4
3,20090312431273200,3,1147,person_click,basic,0,NaN,-413.991405,-159.314686,380.0,494.0,NaN,I gotta run to my meeting!,gramps,tunic.historicalsociety.closet,tunic.historicalsociety.closet.gramps.intro_0_...,0,0,1,0-4
4,20090312431273200,4,1863,person_click,basic,0,NaN,-412.991405,-159.314686,381.0,494.0,NaN,"Can I come, Gramps?",gramps,tunic.historicalsociety.closet,tunic.historicalsociety.closet.gramps.intro_0_...,0,0,1,0-4


**Hover Duration Features Mean (session_id, event)** 

In [4]:
# Hover duration by session id and event
d1 = data[['session_id','level_group','hover_duration','event_name']]

In [5]:
d1 = d1[d1['hover_duration'].notnull()]

In [6]:
hover_mean = d1.groupby(by=['session_id','level_group','event_name'])['hover_duration'].mean()

In [7]:
map = hover_mean.xs('map_hover', level='event_name').reset_index()
object = hover_mean.xs('object_hover', level='event_name').reset_index()

In [8]:
map.rename(columns={'hover_duration': 'map_hover_duration'}, inplace=True)
object.rename(columns={'hover_duration': 'object_hover_duration'}, inplace=True)

In [9]:
d1 = pd.merge(map,object,on=['session_id','level_group'])

In [10]:
d1.head()

,session_id,level_group,map_hover_duration,object_hover_duration
0,20090312431273200,0-4,129.500000,4649.500000
1,20090312431273200,13-22,257.000000,1592.307692
2,20090312431273200,5-12,281.444444,1264.142857
3,20090312433251036,0-4,284.333333,2035.400000
4,20090312433251036,13-22,587.236559,1095.621212


**Bingo Features**

In [11]:
d2 = data[['session_id','level_group','text_fqid','elapsed_time']]
d2 = d2[d2['text_fqid'].notnull()]

In [12]:
d2 = d2[d2['text_fqid'].str.endswith('bingo')]

In [13]:
first_bingo = d2.groupby(['session_id','level_group']).first().reset_index()

In [14]:
first_bingo_elapsed_time = first_bingo[['session_id','level_group','elapsed_time']]

In [15]:
first_bingo_elapsed_time.head()

,session_id,level_group,elapsed_time
0,20090312431273200,13-22,1219035
1,20090312431273200,5-12,346295
2,20090312433251036,13-22,2317036
3,20090312433251036,5-12,453175
4,20090312455206810,13-22,1090365


In [16]:
d2['time_diff']=d2['elapsed_time'].diff()

In [17]:
d2 = d2[d2['time_diff'] >= 0]

In [18]:
bingo_mean = d2.groupby(by=['session_id','level_group'])['time_diff'].mean().to_frame()

In [19]:
bingo_mean.rename(columns={'time_diff': 'bingo_time_mean'}, inplace=True)

In [20]:
d2 = pd.merge(bingo_mean,first_bingo_elapsed_time,on=['session_id','level_group'])

In [21]:
d2.rename(columns={'elapsed_time': 'first_bingo_elapsed_time'}, inplace=True)

In [22]:
d2.head()

,session_id,level_group,bingo_time_mean,first_bingo_elapsed_time
0,20090312431273200,13-22,193390.250000,1219035
1,20090312431273200,5-12,20302.714286,346295
2,20090312433251036,13-22,292371.700000,2317036
3,20090312433251036,5-12,49125.428571,453175
4,20090312455206810,13-22,137567.750000,1090365


**Room Change Feature: Ratio of room change per session id**

In [23]:
d3 = data[['session_id','level_group','room_fqid']]

In [24]:
d3['shift'] = d3['room_fqid'].shift()

/tmp/ipykernel_47/516194630.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  d3['shift'] = d3['room_fqid'].shift()


In [25]:
d3 = d3.groupby(['session_id','level_group']).apply(lambda x: x.iloc[1:]).reset_index(drop=True)

In [26]:
d3 = d3[d3['shift'].notnull()]

In [27]:
d3['room_change']= np.where(d3['room_fqid']==d3['shift'],0,1)

In [28]:
d3_count = d3.groupby(by = ['session_id','level_group'])['room_change'].sum().to_frame()

In [29]:
d3_count.head()

room_change
session_id        level_group             
20090312431273200 0-4                   12
                  13-22                 30
                  5-12                  21
20090312433251036 0-4                    6
                  13-22                 80

In [30]:
d3_total = d3.groupby(by=['session_id','level_group'])['room_change'].count().to_frame()

In [31]:
d3 = pd.merge(d3_count,d3_total,on=['session_id','level_group'])

In [32]:
d3['room_change_ratio']=d3['room_change_x']/d3['room_change_y']

In [33]:
d3.reset_index(inplace = True)

In [34]:
d3.head()

,session_id,level_group,room_change_x,room_change_y,room_change_ratio
0,20090312431273200,0-4,12,164,0.073171
1,20090312431273200,13-22,30,419,0.071599
2,20090312431273200,5-12,21,295,0.071186
3,20090312433251036,0-4,6,138,0.043478
4,20090312433251036,13-22,80,1296,0.061728


In [35]:
d3 = d3[['session_id','level_group','room_change_ratio']]

In [36]:
d3.head()

,session_id,level_group,room_change_ratio
0,20090312431273200,0-4,0.073171
1,20090312431273200,13-22,0.071599
2,20090312431273200,5-12,0.071186
3,20090312433251036,0-4,0.043478
4,20090312433251036,13-22,0.061728


**text_fqid change feature: Ratio of text_fqid change per session id**

In [37]:
d4 = data[['session_id','level_group','text_fqid']]

In [38]:
d4 = d4[d4['text_fqid'].notnull()]

In [39]:
d4['shift'] = d4['text_fqid'].shift()

In [40]:
d4 = d4.groupby(['session_id','level_group']).apply(lambda x: x.iloc[1:]).reset_index(drop=True)

In [41]:
d4 = d4[d4['shift'].notnull()]

In [42]:
d4['text_change']= np.where(d4['text_fqid']==d4['shift'],0,1)

In [43]:
d4_count = d4.groupby(by = ['session_id','level_group'])['text_change'].sum().to_frame()
d4_total = d4.groupby(by= ['session_id','level_group'])['text_change'].count().to_frame()

In [44]:
d4 = pd.merge(d4_count,d4_total,on=['session_id','level_group'])

In [45]:
d4['text_change_ratio']=d4['text_change_x']/d4['text_change_y']

In [46]:
d4=d4['text_change_ratio'].reset_index()

In [47]:
d4.head()

,session_id,level_group,text_change_ratio
0,20090312431273200,0-4,0.295082
1,20090312431273200,13-22,0.179487
2,20090312431273200,5-12,0.184000
3,20090312433251036,0-4,0.200000
4,20090312433251036,13-22,0.219298


Combine All Features

In [48]:
d2.head()

,session_id,level_group,bingo_time_mean,first_bingo_elapsed_time
0,20090312431273200,13-22,193390.250000,1219035
1,20090312431273200,5-12,20302.714286,346295
2,20090312433251036,13-22,292371.700000,2317036
3,20090312433251036,5-12,49125.428571,453175
4,20090312455206810,13-22,137567.750000,1090365


In [49]:
from functools import reduce
dfs = [d1, d3, d4]
# Merging all DataFrames
features = reduce(lambda left, right: pd.merge(left, right, on=['session_id','level_group'], how='inner'), dfs)

In [50]:
features = pd.merge(features,d2,on=['session_id','level_group'],how='left')

In [51]:
features.head(10)

,session_id,level_group,map_hover_duration,object_hover_duration,room_change_ratio,text_change_ratio,bingo_time_mean,first_bingo_elapsed_time
0,20090312431273200,0-4,129.500000,4649.500000,0.073171,0.295082,NaN,NaN
1,20090312431273200,13-22,257.000000,1592.307692,0.071599,0.179487,193390.250000,1219035.0
2,20090312431273200,5-12,281.444444,1264.142857,0.071186,0.184000,20302.714286,346295.0
3,20090312433251036,0-4,284.333333,2035.400000,0.043478,0.200000,NaN,NaN
4,20090312433251036,13-22,587.236559,1095.621212,0.061728,0.219298,292371.700000,2317036.0
5,20090312433251036,5-12,271.333333,1342.312500,0.071066,0.176471,49125.428571,453175.0
6,20090312455206810,0-4,518.000000,8401.000000,0.054054,0.173333,NaN,NaN
7,20090312455206810,13-22,650.833333,1428.692308,0.088154,0.173653,137567.750000,1090365.0
8,20090312455206810,5-12,453.000000,1311.100000,0.086066,0.152542,20191.428571,478661.0
9,20090313091715820,0-4,941.500000,2307.400000,0.034286,0.215385,NaN,NaN


In [52]:
features.to_csv('features_keyue.csv', index=False)

In [53]:
features.head()

,session_id,level_group,map_hover_duration,object_hover_duration,room_change_ratio,text_change_ratio,bingo_time_mean,first_bingo_elapsed_time
0,20090312431273200,0-4,129.500000,4649.500000,0.073171,0.295082,NaN,NaN
1,20090312431273200,13-22,257.000000,1592.307692,0.071599,0.179487,193390.250000,1219035.0
2,20090312431273200,5-12,281.444444,1264.142857,0.071186,0.184000,20302.714286,346295.0
3,20090312433251036,0-4,284.333333,2035.400000,0.043478,0.200000,NaN,NaN
4,20090312433251036,13-22,587.236559,1095.621212,0.061728,0.219298,292371.700000,2317036.0
